In [51]:
workspace() #hierdurch werden Warnungen und Fehlermeldungen behoben. wieso genau weiß ich nicht.

type state #der Typ state soll einen Zustand beschreiben
    j::Int64 #(Binär-)Zahl des Zustands
    n::Int64 #Anzahl der Gitterplätze
    f::Int64 #Faktor. Ist generell ersteinmal 1. Wird eventuell zu -1 verändert, wenn ein Operator auf den Zustand wirkt.
end

function getStateArray(zustand::state) #diese Funktion empfängt einen Zustand vom Typ state. Für diesen Zustand ermittelt 
    a=zeros(Int64, 2*zustand.n) #sie das zugehörige Array mit den Besetzungszahlen
    number = zustand.j
    counter=1
    while number > 0
        a[counter]=number%2
        number= number%2 == 1 ? Int64(number/2 - 0.5) : number/2
        counter=counter+1
    end
    return a
end


#Die Funktion CreateSpin wirkt auf ein Objekt vom Typ state so wie der Erzeugeroperator auf einen Zustand.
function CreateSpin(i,s,zustand) #i ist der Gitterplatz, s ist der Spin, mit Wert 2 für up und 1 für down
   if isa(zustand,Int64) true
        b=0
    else
   number=zustand.j
   occupationnumber= getStateArray(zustand)[2i-s+1]
   newnumber = number + 2^(2*i-s)
    a=0
    for j in 1:2i-s
            a+= getStateArray(zustand)[j]
    end
   newfactor = (1-occupationnumber)*zustand.f*(-1)^a
   b= newfactor == 0 ? 0:  newstate= state(newnumber,zustand.n,newfactor)
    end
return b
end

#Die Funktion AnnihilateSpin wirkt auf ein Objekt vom Typ state so wie der Vernichteroperator auf einen Zustand.
function AnnihilateSpin(i,s,zustand) #i ist der Gitterplatz, s ist der Spin, mit Wert 2 für up und 1 für down
    if isa(zustand,Int64) true
      b=0
    else
     number=zustand.j
     occupationnumber= getStateArray(zustand)[2i-s+1] 
     a=0
      for j in 1:2i-s
            a+= getStateArray(zustand)[j]
      end
     newfactor = occupationnumber*zustand.f*(-1)^a
     newnumber = number - 2^(2*i-s)
     b= newfactor == 0 ? 0:  newstate= state(newnumber,zustand.n,newfactor)
    end
   return b
end


function Ham(E,U,t,Es,V,n) #E ist die Eigenenergie eines Zustandes. U ist das "Hubbard U". Es wirkt nur bei der Störstelle., t für tight-binding, V für Störstelle-Bad, Es für Störstelle
    H=zeros(4^n,4^n) 
for j in 0:4^n-1
        y=state(j,n,1) 
     i = 1 
#hier beginnt H_imp
          for s in 1:2  #erster Summenterm mit E
            x=CreateSpin(i,s,AnnihilateSpin(i,s,y)) 
           if isa(x,state) true
                H[(x.j)+1,(y.j)+1]+= Es*x.f
            end
          end
            #zweiter Summenterm mit U
        x=CreateSpin(i,2,AnnihilateSpin(i,2,CreateSpin(i,1,AnnihilateSpin(i,1,y))))  
          if isa(x,state) true
               H[(x.j)+1,(y.j)+1]+= U*x.f 
          end  
 #nun beginnt H_imp-bath mit V statt t
           for s in 1:2
            i=1
            while i+1<=n
                x=CreateSpin(i,s,AnnihilateSpin(i+1,s,y)) 
                 if isa(x,state) true
                 H[(x.j)+1,(y.j)+1]+= V*x.f 
                 end
              i+=1  
            end
                    k=1
            while k+1<=n
                x=CreateSpin(k+1,s,AnnihilateSpin(k,s,y))
                 if isa(x,state) true
                 H[(x.j)+1,(y.j)+1]+= t*x.f
                 end
              k+=1  
            end
         end 
i=+1
#Hier beginnt H_bath. U ist null, also habe ich die zugehörige for-schleife entfernt.        
     for i in 2:n         
        for s in 1:2  #erster Summenterm mit E
            x=CreateSpin(i,s,AnnihilateSpin(i,s,y))
           if isa(x,state) true
                H[(x.j)+1,(y.j)+1]+= E*x.f
            end
        end
            
      
   end
            #dritter Summenterm mit t
           for s in 1:2
            i=2
            while i+1<=n
                x=CreateSpin(i,s,AnnihilateSpin(i+1,s,y))
                 if isa(x,state) true
                 H[(x.j)+1,(y.j)+1]+= t*x.f 
                 end
              i+=1  
            end
                    k=2
            while k+1<=n
                x=CreateSpin(k+1,s,AnnihilateSpin(k,s,y))
                 if isa(x,state) true
                 H[(x.j)+1,(y.j)+1]+= t*x.f
                 end
              k+=1  
            end
            end 
      
   
        
end
return H
end

p=Ham(0,5,0,0,0,2)
eigvals(p)



16-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 5.0
 5.0
 5.0
 5.0